# Personality Atlas — Quick Start

**Reproduce the atlas results in under 5 minutes.** No API keys required.

> Raetano, J., Gregor, J., & Tamang, S. (2026). *A Survey and Computational Atlas of Personality Models.* ACM TIST. Under review.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Wildertrek/survey/blob/main/notebooks/atlas_quick_start.ipynb)

---

## 1. Setup (~30 seconds)

In [ ]:
# Clone the atlas repository
!git clone --depth 1 https://github.com/Wildertrek/survey.git atlas
!pip install -q pandas scikit-learn==1.5.0 joblib matplotlib numpy faiss-cpu

## 2. Load and Predict — Any of 44 Models (~10 lines)

In [ ]:
import ast
import numpy as np
import pandas as pd
import joblib

# Pick any model: ocean, hexaco, mbti, mmpi, scid, npi, dt4, tci, ...
SLUG = "ocean"

df = pd.read_csv(f"atlas/datasets/{SLUG}.csv")
embeddings = pd.read_csv(f"atlas/Embeddings/{SLUG}_embeddings.csv")
model = joblib.load(f"atlas/models/{SLUG}_rf_model.pkl")
encoder = joblib.load(f"atlas/models/{SLUG}_label_encoder.pkl")

X = np.array([ast.literal_eval(e) for e in embeddings["Embedding"]])
predictions = encoder.inverse_transform(model.predict(X))
accuracy = (predictions == df["Factor"].values).mean()

print(f"{SLUG.upper()}: {len(df)} traits, {len(set(predictions))} factors, accuracy = {accuracy:.1%}")
print(f"Factors: {sorted(set(predictions))}")

## 3. Atlas Overview — All 44 Models

In [ ]:
import os

slugs = sorted([f.replace(".csv", "") for f in os.listdir("atlas/datasets") if f.endswith(".csv")])
print(f"Atlas contains {len(slugs)} personality models:\n")

results = []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    n_factors = df["Factor"].nunique()
    
    model = joblib.load(f"atlas/models/{slug}_rf_model.pkl")
    enc = joblib.load(f"atlas/models/{slug}_label_encoder.pkl")
    emb = pd.read_csv(f"atlas/Embeddings/{slug}_embeddings.csv")
    X = np.array([ast.literal_eval(e) for e in emb["Embedding"]])
    preds = enc.inverse_transform(model.predict(X))
    acc = (preds == df["Factor"].values).mean()
    
    results.append({"Model": slug.upper(), "Traits": len(df), "Factors": n_factors, "Accuracy": f"{acc:.1%}"})

results_df = pd.DataFrame(results).sort_values("Accuracy", ascending=False)
results_df.index = range(1, len(results_df) + 1)
results_df

## 4. Reproduce PCA — Cross-Model Dimensionality Analysis (Paper Figure 5-8)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Category assignments for the 7-category taxonomy
CATEGORIES = {
    "Trait-Based": ["ocean", "hexaco", "mbti", "epm", "16pf", "ft"],
    "Narcissism-Based": ["npi", "pni", "hsns", "dtm", "dt4", "ffni", "ffnisf", "narq", "mcmin", "ipn"],
    "Motivational/Value": ["stbv", "sdt", "rft", "aam", "mst", "cs"],
    "Cognitive/Learning": ["pct", "cest", "scm", "fsls"],
    "Clinical/Health": ["mmpi", "scid", "bdi", "gad7", "wais", "tci", "mcmi", "tmp", "rit", "tat"],
    "Interpersonal/Conflict": ["disc", "tki"],
    "Application-Specific": ["riasec", "cmoa", "tei", "bt", "em", "papc"]
}
slug_to_cat = {s: c for c, slugs in CATEGORIES.items() for s in slugs}

# Load all embeddings
all_vecs, all_labels, all_cats = [], [], []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    emb = pd.read_csv(f"atlas/Embeddings/{slug}_embeddings.csv")
    X = np.array([ast.literal_eval(e) for e in emb["Embedding"]])
    all_vecs.append(X)
    all_labels.extend([slug.upper()] * len(X))
    all_cats.extend([slug_to_cat.get(slug, "Unknown")] * len(X))

X_all = np.vstack(all_vecs)
print(f"Loaded {X_all.shape[0]} embeddings ({X_all.shape[1]}-dim) from {len(slugs)} models")

In [ ]:
# PCA — Scree plot (Paper Figure 5)
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_all)
cumvar = np.cumsum(pca.explained_variance_ratio_) * 100

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(range(1, 51), pca.explained_variance_ratio_ * 100, alpha=0.6, label="Individual")
ax.plot(range(1, 51), cumvar, "r-o", markersize=3, label="Cumulative")
ax.axhline(y=cumvar[-1], color="gray", linestyle="--", alpha=0.5)
ax.set_xlabel("Principal Component")
ax.set_ylabel("Variance Explained (%)")
ax.set_title(f"PCA Scree Plot — {X_all.shape[0]} Trait Embeddings from 44 Models\n50 PCs capture {cumvar[-1]:.1f}% of variance")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# PCA — Model centroids in PC1-PC2 space (Paper Figure 6)
cat_colors = {
    "Trait-Based": "#1f77b4", "Narcissism-Based": "#ff7f0e",
    "Motivational/Value": "#2ca02c", "Cognitive/Learning": "#d62728",
    "Clinical/Health": "#9467bd", "Interpersonal/Conflict": "#8c564b",
    "Application-Specific": "#e377c2"
}

centroids = pd.DataFrame({
    "PC1": X_pca[:, 0], "PC2": X_pca[:, 1],
    "Model": all_labels, "Category": all_cats
}).groupby(["Model", "Category"])[["PC1", "PC2"]].mean().reset_index()

fig, ax = plt.subplots(figsize=(12, 8))
for cat, color in cat_colors.items():
    subset = centroids[centroids["Category"] == cat]
    ax.scatter(subset["PC1"], subset["PC2"], c=color, s=80, label=cat, alpha=0.8, edgecolors="white", linewidth=0.5)
    for _, row in subset.iterrows():
        ax.annotate(row["Model"], (row["PC1"], row["PC2"]), fontsize=7, alpha=0.7,
                    xytext=(4, 4), textcoords="offset points")

ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
ax.set_title("44 Personality Models in PC1-PC2 Space (Model Centroids)")
ax.legend(loc="best", fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
# PCA — All traits colored by category (Paper Figure 8)
fig, ax = plt.subplots(figsize=(12, 8))
for cat, color in cat_colors.items():
    mask = [c == cat for c in all_cats]
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1], c=color, s=4, alpha=0.3, label=cat)

ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
ax.set_title(f"All {X_all.shape[0]} Trait Embeddings — 44 Models, 7 Categories")
ax.legend(markerscale=5, fontsize=8)
plt.tight_layout()
plt.show()

## 5. Cross-Model Search with FAISS

In [ ]:
import faiss

# Build FAISS index over entire atlas
X_norm = X_all / np.linalg.norm(X_all, axis=1, keepdims=True)
index = faiss.IndexFlatIP(X_norm.shape[1])
index.add(X_norm.astype(np.float32))

# Build metadata for lookups
all_factors = []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    all_factors.extend(df["Factor"].values)

print(f"FAISS index: {index.ntotal} vectors, {X_norm.shape[1]}-dim")
print(f"Ready for cross-model personality search")

In [ ]:
# Query: find traits similar to "anxiety" across all 44 models
# Use an existing anxiety-related embedding as the query
query_slug = "gad7"
query_df = pd.read_csv(f"atlas/datasets/{query_slug}.csv")
query_emb = pd.read_csv(f"atlas/Embeddings/{query_slug}_embeddings.csv")
# Pick first trait as query
q = np.array([ast.literal_eval(query_emb["Embedding"].iloc[0])]).astype(np.float32)
q = q / np.linalg.norm(q)

D, I = index.search(q, 20)

query_trait = query_df.iloc[0]
print(f"Query: {query_slug.upper()} / {query_trait['Factor']} — \"{query_trait['Adjective']}\"\n")
print(f"{'Rank':<5} {'Model':<10} {'Factor':<35} {'Category':<22} {'Score':.5}")
print("-" * 85)
for rank, (idx, score) in enumerate(zip(I[0], D[0]), 1):
    print(f"{rank:<5} {all_labels[idx]:<10} {all_factors[idx]:<35} {all_cats[idx]:<22} {score:.4f}")

## 6. Lexical Schema — What's in Each Model

In [ ]:
# Inspect any model's lexical schema
INSPECT = "hexaco"  # change to any slug

df = pd.read_csv(f"atlas/datasets/{INSPECT}.csv")
print(f"{INSPECT.upper()}: {len(df)} traits across {df['Factor'].nunique()} factors\n")

for factor, group in df.groupby("Factor"):
    sample = group.head(3)
    adjectives = ", ".join(sample["Adjective"].values)
    print(f"  {factor} ({len(group)} traits): {adjectives}, ...")

---

**Repository:** [github.com/Wildertrek/survey](https://github.com/Wildertrek/survey)  
**Paper:** Raetano, J., Gregor, J., & Tamang, S. (2026). *A Survey and Computational Atlas of Personality Models.* ACM TIST.  
**License:** MIT